In [1]:
#%pip install -U scikit-learn
#%pip install -U category-encoders
#%pip install -U seaborn

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from psycopg2 import sql 
from psycopg2.extensions import AsIs
import numpy as np
import yaml
import re
import sys
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
import sklearn.model_selection as ms
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
import category_encoders as ce # sklearn related
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from matplotlib.patches import BoxStyle
import seaborn as sns
import time
import timeit

from scipy.stats import yeojohnson
# makes it so all modules are reloaded, allowing...
# ...me to not restart the kernel when they are edited!
%load_ext autoreload 
%autoreload 2 
# Custom import(s)
import Utility_methods as ums
from CallPostgre import Database 
from ML_models import SupervisedLearning

In [1]:
def clean_dataframes_data(dfs_to_merge):
    for df in dfs_to_merge:
        # From a quick look at dataFrame.dtypes, it seems only the date 
        #  (listing_date), althought already in the right format, and the
        #  home divisions (float64 instead of int64) are mistyped,
        #  so let's fix that
        for column_name in df.columns:
            #----------------
            #Type corrections:
            #----------------
            #Int64 corrections#
            # Note: due to numpy compatibility, pandas cannot store features with
            #  missing values (NaNs) as int, therefore it stores them as floats
            #  a workaround is casting the column to "Nullable Integer Data
            #  Type: Int64"
            #
            # Nontheless, Int64Dtype is still not supported in some operations
            #  such as df[column_name].dtype, giving errors such as:
            #  "Cannot interpret 'Int64Dtype()' as a data type". 
            # According to  an issue open on github, the recommendation is to
            #  keep integers columns with "nans" as floats, since despite not
            #  having an ideal representation, it can perform the same operations
            #  as if it were "int".
            #
            # If this gets fixed, uncomment the Int64 convertion code bellow
            #  to use it as such.

            #!Uncomment once Int64Dtype is integrated as the other dtypes are
            #
            # Check any "counter" column ending in "_count"
            #counts_regex = re.compile("_count$")
            #if re.search(counts_regex, column_name): 
            #    #If the counter column is in float instead of int, convert
            #    if np.issubdtype(df[column_name].dtype, float):
            #        # Note astype('int') doesn't work when there's NaNs present
            #        df[column_name] = df[column_name].astype('Int64') 

            #e# Switch to "elif" if the above is uncommented
            #
            #* An unproperly casted datetime would only be type objects so only
            #  those need to be checked
            if np.issubdtype(df[column_name].dtype , object): #* 
                row_number = 0
                # Check type of the first element of each column(if NaN check 
                #  the one after)
                value = df[column_name].iloc[row_number] 
                if isinstance(value, list):
                    # Avoid the "The truth value of an array with more than one
                    #  one element is ambiguous" error from using `.isna()` on list
                    #
                    # Features with lists are guaranteed to not be nan as the
                    #  scrapping populates an "empty list" therefore, "value"
                    #  will never be NaN in feature columns with lists, and the
                    #  following `.isna()` check is thus protected from the 
                    #  aforementioned error.)
                    continue
                # Ignore nans before checking type
                while(pd.isna(value)): 
                    row_number += 1
                    value = df[column_name].iloc[row_number]
                
                ## Datetime corrections ##
                #
                # Convert columns which contain pandas datetime64 fomat #
                #  YYYY-MM-DD from "object" to "datetime64"
                #
                # Validates days and months but not years so this code can be
                #  used centuries in the future (or past if we travel there
                #  someday! :)) 
                datetime_regex = re.compile("^[^\d]*(\d{4}-(1[0-2]|0[1-9])-(3[01]|[1-2][0-9]|0[1-9]))")
                if re.match(datetime_regex, str(value)):
                    df[column_name] = pd.to_datetime(df[column_name])
            
                    #------------------------
                    ## Feature engineering ##
                    # (datetime)
                    #------------------------
                    #
                    # Datetimes to [year] [month] [day]
                    df[column_name+'_year'] = df[column_name].dt.year
                    df[column_name+'_month'] = df[column_name].dt.month
                    df[column_name+'_day'] = df[column_name].dt.day
                    # Discard the original datetime afterwards
                    df.drop(columns=column_name, inplace=True)
    
    cleaned_dfs_to_merge = dfs_to_merge
    return cleaned_dfs_to_merge

def integrate_dataframes_data(cleaned_dfs_to_merge):
    # Edit if more tables are added in the future
    df_real_estate_agent, df_home_liverpool = cleaned_dfs_to_merge[0], cleaned_dfs_to_merge[1]
    integrated_df= pd.merge(df_real_estate_agent, df_home_liverpool,
                        on='real_estate_agent_id',
                        how='right'
    )
    return integrated_df


def split_dataframe_data_train_test(integrated_df, target_variable_name, is_nan_splitting):
    predictors_df = integrated_df.drop(columns=target_variable_name, inplace=False)
    target_df = integrated_df[target_variable_name]

    if is_nan_splitting:
        # Splitting the dataset by nans in the target variable 
        test_df = integrated_df[integrated_df[target_variable_name].isna()]
        X_train = predictors_df.drop(test_df.index, inplace=False) 
        y_train = target_df.drop(test_df.index, inplace=False) 
        X_test =  test_df.drop(columns=target_variable_name, inplace=False)
        y_test = test_df[target_variable_name]
    else:
        # Splitting the dataset by randomly sampling
        #  (other methods can be used)
        X_train, X_test, y_train, y_test = ms.train_test_split(predictors_df,
                                                               target_df,
                                                               test_size=0.20,
                                                               random_state=30) 

    # If single-target prediction, convert to (single column) dataframe format
    #  for ease of use:
    if type(y_train) == pd.Series:
        y_train = y_train.to_frame()
    if type(y_test) == pd.Series:
        y_test = y_test.to_frame()
        
    return X_train, X_test, y_train, y_test 



def handle_missing_dataframe_data_X(X_train, X_t, missing_data_threshold,
                                    features_to_drop_expert_knowledge,
                                    numeric_imputation_method_X,
                                    categorical_imputation_method_X):
    # Note: X_train mus thave the same features has X_test, so if expert knowledge
    #        or missing data thresholds remove a feature in X_train, then it must
    #        also be removed in X_test

    # Drop columns not deemed useful by an expert
    for column_to_drop_expert_knowledge in features_to_drop_expert_knowledge:
        if column_to_drop_expert_knowledge in X_t.columns:
            X_t=X_t.drop(columns=[column_to_drop_expert_knowledge])
            X_train=X_train.drop(columns=[column_to_drop_expert_knowledge]) 
            
    # Keep only columns with "less than threshold" ratio of missing values 
    X_t_to_fill = X_t[X_t.columns[(X_train.isna().p_mean()<missing_data_threshold)]] 
    X_train = X_train[X_train.columns[(X_train.isna().mean()<missing_data_threshold)]] 

    # Split Dataframe by type #
    #
    # Check type hierarchy at (github): numpy/numpy/core/numerictypes.py, line ~40 
    # Check https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
    #  for deprecated numpy.type alliases
    X_t_numbers_no_dates_times, X_t_numbers_dates_times, X_t_objects, X_t_bools, X_t_lists = ums.split_dataframe_data_by_type(X_t_to_fill)
    # Have the trainset values as base for the preprocessing measures of both
    #  train and test
    X_train_numbers_no_dates_times, X_train_numbers_dates_times, X_train_objects, X_train_bools, X_train_lists = ums.split_dataframe_data_by_type(X_train) 
    
    if X_t_numbers_no_dates_times.empty:
        imputed_X_t_numbers_no_dates_times = X_t_numbers_no_dates_times
    else:    
        # Numeric missing data handling 
        # This excludes years/months/days etc 
        match(numeric_imputation_method_X):      
            case 'DEL': # delete observations with missing data
                imputed_X_t_numbers_no_dates_times = X_t_numbers_no_dates_times.dropna(inplace=False)
                # Drop indexes of other data types so they don't impute on indexes
                #  which no longer exist on this one
                X_t_objects, X_t_bools, X_t_numbers_dates_times, X_t_lists = ums.drop_indexes_of_other_data_types(imputed_X_t_numbers_no_dates_times, \
                                                                                                                  X_t_objects, \
                                                                                                                  X_t_bools, \
                                                                                                                  X_t_numbers_dates_times, \
                                                                                                                  X_t_lists)
        
            case 1: # Mean
                #* Default is np.array, convert to df with transform='pandas'
                mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean').set_output(transform='pandas') #*
                mean_imputer.fit(X_train_numbers_no_dates_times) 
                imputed_X_t_numbers_no_dates_times = mean_imputer.transform(X_t_numbers_no_dates_times)
            
            case 2: #Median
                median_imputer = SimpleImputer(missing_values=np.nan, strategy='median').set_output(transform='pandas')
                median_imputer.fit(X_train_numbers_no_dates_times) 
                imputed_X_t_numbers_no_dates_times = median_imputer.transform(X_t_numbers_no_dates_times)

            case 3: #Ffill
                # EXPERIMENTAL as it's not applicable on the test set the same
                #  way as in the training set. Also (I think) would work better
                #  if the dataframe could be sorted by 'price' before being
                #  sort back to id (a little touch I think will make ffill 
                #  better, in other words, sorting by a row that is 
                #  proportionate to the features being filled). 
               
                imputed_X_t_numbers_no_dates_times = X_t_numbers_no_dates_times.ffill(inplace=False)
                # In case there's NaNs in the initial row(s), fill them with
                #  "backwawrd fill"
                imputed_X_t_numbers_no_dates_times.bfill(inplace=True, limit=1)
               
    # Categorical missing data handling #
    if X_t_objects.empty:
        imputed_X_t_objects = X_t_objects
    else: 
       match(categorical_imputation_method_X):
           case 'DEL': # delete observations with missing data
               imputed_X_t_objects = X_t_objects.dropna(inplace=False)
               # Drop indexes of other data types so they don't impute on indexes
               #  which no longer exist on this one
               imputed_X_t_numbers_no_dates_times, X_t_bools, X_t_numbers_dates_times, X_t_lists = ums.drop_indexes_of_other_data_types(imputed_X_t_objects,  
                                                                                                                                        imputed_X_t_numbers_no_dates_times, 
                                                                                                                                        X_t_bools, 
                                                                                                                                        X_t_numbers_dates_times, 
                                                                                                                                        X_t_lists)  

        
           case 'A': #Mode imputation
               mean_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent').set_output(transform='pandas')
               mean_imputer.fit(X_train_objects)
               imputed_X_t_objects = mean_imputer.transform(X_t_objects)
  

    
    # Boolean missing data handling #
    #
    # Not required so far
    imputed_X_t_bools = X_t_bools 
    # If option is 'DEL', drop indexes of  so they don't impute on,
    #  or have got imputed, indexes which no longer exist on this one.
    #  Can use:
    #  `imputed_X_t_numbers_no_dates_times, X_t_numbers_dates_times, 
    #  X_t_bools, X_t_lists = 
    #  ums.drop_indexes_of_other_data_types(imputed_X_t_objects, 
    #                                        imputed_X_t_numbers_no_dates_times, \
    #                                        X_t_numbers_dates_times, \
    #                                        X_t_bools, \
    #                                        X_t_lists)  
       
    # Datetime missing data handling (for interpolation) #
    #
    # Not required so far
    # Also drop indexes of other types if the option is 'DEL'
    imputed_X_t_numbers_dates_times = X_t_numbers_dates_times
    
    # List missing data handling
    #
    # Not required so far
    # Each list-type feature handling might require a tailored approach
    # Also drop indexes of other types if the option is 'DEL'
    imputed_X_t_lists = X_t_lists 
 
   
    #Rebuild the dataset
    imputed_X_t = pd.concat([imputed_X_t_numbers_no_dates_times,
                             imputed_X_t_numbers_dates_times, imputed_X_t_objects,
                             imputed_X_t_bools, imputed_X_t_lists],
                            axis=1)
    
    if imputed_X_t.isna().p_sum().sum() == 0:
        print(f"No missing values left. Imputation successful (X)!")
    
    # All train_test_split indexes are preserved up to this point
    return imputed_X_t


def handle_missing_dataframe_data_y(y_train, numeric_imputation_method_y, categorical_imputation_method):
    # Split target(s) Dataframe by type ##
    #
    # Shouldn't be necessary for single-target prediction, but this way the algorithm stays more general.
    # Check type hierarchy at (github): numpy/numpy/core/numerictypes.py, line ~40 
    # Check https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations for deprecated numpy.type alliases
    y_train_numbers_no_dates_times, y_train_numbers_dates_times, y_train_objects, y_train_bools, y_train_lists = ums.split_dataframe_data_by_type(y_train)


    # Numeric missing data handling #
    # This excludes years/months/days etc 
    #
    # If empty no point in imputing
    if y_train_numbers_no_dates_times.empty:
        imputed_y_train_numbers_no_dates_times = y_train_numbers_no_dates_times
    else:
        match(numeric_imputation_method_y):  
            case 'DEL': # delete observations with missing data
                 imputed_y_train_numbers_no_dates_times =  y_train_numbers_no_dates_times.dropna(inplace=False)
                 # (Only relevant if multi-target) Drop indexes of other data
                 #  types so they don't impute on indexes which no longer exist
                 #  on this one
                 y_train_objects, y_train_bools, y_train_numbers_dates_times, y_train_lists = ums.drop_indexes_of_other_data_types(imputed_y_train_numbers_no_dates_times, \
                                                                                                                                   y_train_objects, \
                                                                                                                                   y_train_bools, \
                                                                                                                                   y_train_numbers_dates_times, \
                                                                                                                                   y_train_lists)                
                         
            case 1: # Mean
                # Pandas method is fine as there is no data-leakege prevention 
                #  concerns (y_test won't be imputed :))
                imputed_y_train_numbers_no_dates_times = y_train_numbers_no_dates_times.fillna(y_train_numbers_no_dates_times.mean()) 
    
            case 2: # Median
                imputed_y_train_numbers_no_dates_times = y_train_numbers_no_dates_times.fillna(y_train_numbers_no_dates_times.median())
            
            case 3: # Ffill
                imputed_y_train_numbers_no_dates_times = y_train_numbers_no_dates_times.ffill(inplace=False)
                # In case there's NaNs in the first rows, bfill them
                imputed_y_train_numbers_no_dates_times.bfill(inplace=True, limit=1)


    if y_train_objects.empty:
        imputed_y_train_objects = y_train_objects
    else:   
        # Categorical missing data handling #
        match(categorical_imputation_method):
           case 'DEL': #delete observations with missing data
               imputed_y_train_objects = y_train_objects.dropna(inplace=False)
               # (Only relevant if multi-target) Drop indexes of other data
                 #  types so they don't impute on indexes which no longer exist
                 #  on this one
               imputed_y_train_numbers_no_dates_times, y_train_bools, y_train_numbers_dates_times, y_train_lists = ums.drop_indexes_of_other_data_types(imputed_y_train_objects, \
                                                                                                                                                        imputed_y_train_numbers_no_dates_times, \
                                                                                                                                                        y_train_bools, \
                                                                                                                                                        y_train_numbers_dates_times, \
                                                                                                                                                        y_train_lists)                
                
           case 'A': #Mode imputation
               imputed_y_train_objects = y_train_objects.fillna(y_train_objects.mode().iloc[0])

    # Boolean missing data handling #
    #
    # Not required so far
    # Also drop indexes of other types if the option is 'DEL'
    imputed_y_train_bools = y_train_bools #

    
    # Datetime missing data handling (for interpolation) #
    #
    # Not required so far
    # Also drop indexes of other types if the option is 'DEL'
    imputed_y_train_numbers_dates_times = y_train_numbers_dates_times

    
    # List missing data handling
    #
    # Not required so far
    # Each list-type feature handling might require a tailored approach
    # Also drop indexes of other types if the option is 'DEL'
    imputed_y_train_lists = y_train_lists 
    
    #Rebuild the dataset
    imputed_y_train = pd.concat([imputed_y_train_numbers_no_dates_times, 
                                 imputed_y_train_numbers_dates_times,
                                 imputed_y_train_objects, imputed_y_train_bools,
                                 imputed_y_train_lists], 
                                axis=1)
    
    if imputed_y_train.isna().sum().sum() == 0:
        print(f"No missing values left. Imputation successful! (y)")

    return imputed_y_train


def detect_outliers(X_t, column_name, distribution_type, features_outlier_limits, features_in_wrong_distribution):
    skewness = X_t[column_name].skew()
    
    # Based on a quick search, it seems the sknewness values mean:
    #     Between [-0.5 , 0.5] -> fairly symmetrical, 'normal' distribution
    #     Between [-1, -0.5] or [0.5, 1] -> fairly skewed, 'skewed' distrib.
    #     <-1 or >1 -> very skewed -> 'skewed' (other?) distrib.
        
    match(distribution_type):
        case 'normal': # Z-score
            if not -0.5 < skewness < 0.5:
                print(f"\nWarning: {column_name} is detecting outliers\n"
                      f"using a method more indicated for normal distributions.\n"
                      f"A skewness value of {skewness:.5f} indicates it is a skewed\n"
                      f"distribution, better suited for Interquantile Range.\n" 
                      f"If you're sure you want it treated this way, comment\n" 
                      f"the '.append()' and 'return' lines.\n" 
                      f"It will be skipped for this method otherwise.\n")
                #c#features_in_wrong_distribution.append(column_name)  
                #c#return features_outlier_limits, features_in_wrong_distribution

            # Calculate min/max values for acceptable Z-score #
            #
            #* 99.7% of the values either way. Edit at will the acceptable
            #   thresholds.
            zscores = [-3,3] #* 
            standard_deviation = X_t[column_name].std()
            mean = X_t[column_name].mean()
            min_nonoutlier_value = zscores[0]*standard_deviation + mean
            max_nonoutlier_value = zscores[1]*standard_deviation + mean

            features_outlier_limits[column_name] = [min_nonoutlier_value, max_nonoutlier_value]
            
            # Plot
            #p#ums.plot_distribution_zscore(X_t, column_name, zscores, mean,
            #                               standard_deviation, 
            #                               min_nonoutlier_value,
            #                               max_nonoutlier_value)
            
    
        case 'skewed': # IQR
            if -0.5 < skewness < 0.5:
                print(f"\nWarning: {column_name} is detecting outliers\n"
                    f"using a method more indicated for skewed distributions.\n"
                    f"A skewness value of {skewness:.5f} indicates it is a normal\n"
                    f"distribution, better suited for Z-score.\n" 
                    f"If you're sure you want it treated this way, comment\n" 
                    f"the '.append()' and 'return' lines.\n" 
                    f"It will be skipped for this method otherwise.\n")
                # Note: if the column is, in this moment, removed from 
                #       'features_to_remove_outliers_from', since that is the object
                #       we iterating over, it will "end" the current loop and it 
                #       will also skip the next variable (why?), dangerous!
                #
                # Save the variable to safely remove it from the list later
                features_in_wrong_distribution.append(column_name)  
                return features_outlier_limits, features_in_wrong_distribution

            #Calculate 25/75th percentiles, IQR, and limits
            Q1 = X_t[column_name].quantile(0.25)
            Q3 = X_t[column_name].quantile(0.75)
            IQR = Q3-Q1
            min_nonoutlier_value = Q1 - 1.5*IQR
            max_nonoutlier_value = Q3 + 1.5*IQR

            #p#print(f"Feature: {column_name}")
            #p#print(f"Q1: {Q1}\nQ3: {Q3}\nIQR: {IQR}")
            #p#print(f"Min. non-outlier value: {min_nonoutlier_value}")
            #p#print(f"Max. non-outlier value: {max_nonoutlier_value}")

            features_outlier_limits[column_name] = [min_nonoutlier_value, max_nonoutlier_value]
            
            # Plot
            #p#ums.plot_distribution_IQR(X_t, column_name)


        case 'other': #Percentile threshold
            # Warning: IQR can go below the lowest number since it has a "-1.5IQR"
            #         term but percentile cannot. Therefore if there are a lot of
            #         minimum or maximum values (such as many 1's in 
            #         living_room/bathroom counts), the percentile's limit will
            #         be at those values, deeming them outliers.
            #         Use with care if maximum and minimum values in any feature
            #         being analysed repeat themselves a lot.
            #
            # Will remove a big percentages of variables whose extremes are 
            #  very populated, such as "1" in the '__room_count' variables
            percentile_threshold = 0.01  
            min_nonoutlier_value = X_t[column_name].quantile(percentile_threshold)
            max_nonoutlier_value = X_t[column_name].quantile(1-percentile_threshold)

            #p#print(f"Feature: {column_name}")
            #p#print(f"Min. non-outlier value: {min_nonoutlier_value}")
            #p#print(f"Max. non-outlier value: {max_nonoutlier_value}")
            
            features_outlier_limits[column_name] = [min_nonoutlier_value, max_nonoutlier_value]

            # Plot
            #p#ums.plot_distribution_percentile(X_t, column_name, 
            #                                  percentile_threshold,
            #                                  min_nonoutlier_value,
            #                                  max_nonoutlier_value)
            
    return features_outlier_limits, features_in_wrong_distribution


def treat_outliers(removed_outliers_X_t, outlier_treating_method, features_to_remove_outliers_from, features_outlier_limits):

    match(outlier_treating_method):
        case 0: # Custom method # Ignores distribution_type
            #
            # Note: these still represent "natural" possibilities within the home
            #       population of the city in question. They will be removed for
            #       testing purposes nontheless.
            #
            # Remove the largest price, which had a decent gap to the second largest
            column_name = 'price'
            if column_name in removed_outliers_X_t.columns:
                removed_outliers_X_t = removed_outliers_X_t.drop(index=removed_outliers_X_t['price'].nlargest(n=1).index)
            # Remove the northenmost latitude (coordinate_x) value which had a
            #  decent gap to the second largest
            column_name = 'coordinate_x'
            if column_name in removed_outliers_X_t.columns:
                removed_outliers_X_t = removed_outliers_X_t.drop(index=removed_outliers_X_t['coordinate_x'].nlargest(n=1).index)
            # There's only a few id's below "62000000", lowest being in the
            # 48000000's which will affect feature scaling, so they'll be dropped.
            column_name = 'home_liverpool_id'
            if column_name in removed_outliers_X_t.columns:
                removed_outliers_X_t = removed_outliers_X_t.drop(index=removed_outliers_X_t[removed_outliers_X_t['home_liverpool_id']<62000000].index)

            word = 'expert-elicited'
            
    
        case 1: # Trimming
            for column_name in features_to_remove_outliers_from:
                removed_outliers_X_t = removed_outliers_X_t[ (removed_outliers_X_t[column_name]>features_outlier_limits[column_name][0]) 
                                                             &
                                                             (removed_outliers_X_t[column_name]<features_outlier_limits[column_name][1]) ]
                word = 'trimmed'
                
                
        case 2: # Capping
            for column_name in features_to_remove_outliers_from:
                # Cap min
                removed_outliers_X_t[column_name] = np.where(removed_outliers_X_t[column_name]<features_outlier_limits[column_name][0],
                                                             features_outlier_limits[column_name][0],
                                                             removed_outliers_X_t[column_name])    
                # Cap max
                removed_outliers_X_t[column_name] = np.where(removed_outliers_X_t[column_name]>features_outlier_limits[column_name][1],
                                                             features_outlier_limits[column_name][1],
                                                             removed_outliers_X_t[column_name]) 

                word = 'capped'
                
    '''
    # Post outlier treatment plot of the suitable features
    for column_name in features_to_remove_outliers_from:
        plot1 = sns.histplot(removed_outliers_X_t[column_name], bins=30, edgecolor='black', kde=True)
        plot1.set_xlabel(f"{column_name}")
        plot1.set_ylabel(f"Count")
        plot1.set_title(f"{column_name} histogram, ({word} outliers)", y=1.02)
        plt.show()
        plot1.figure.savefig(f"{word}_outliers_{column_name}")
    '''
    
    return removed_outliers_X_t


def remove_outliers(X_t, distribution_type, outlier_treating_method, features_to_remove_outliers_from='all'):
    
    if features_to_remove_outliers_from =='all':
        features_to_remove_outliers_from = X_t.columns

    
    numeric_and_not_id_date_time_column = []
    for column_name in features_to_remove_outliers_from:
        # It usually doesnt make sense for date/time measures like months and days
        #  to have outliers. Additionally, in the context of the collected listings
        #  data, neither does it for 'year'
        # Ignore non numeric features and dates/times
        if np.issubdtype(X_t[column_name], np.number) and not (ums.is_date_or_time_column_name(column_name) or ums.is_year_column_name(column_name)):
            numeric_and_not_id_date_time_column.append(column_name)
    features_to_remove_outliers_from = numeric_and_not_id_date_time_column
    
    
    # If we detect and remove outliers of a feature in the same iteration
    #  the next iteration (feature) will have its detection influenced by the
    #  removal of the values of the first. Therefore the limits will be simply
    #  stored and the outliers removed alltogether at the end.
    features_outlier_limits={}
    features_in_wrong_distribution=[]
    for column_name in features_to_remove_outliers_from:    
        #------------------
        # Detect outliers (max and min values) #
        #------------------    
        features_outlier_limits, features_in_wrong_distribution = detect_outliers(X_t,
                                                                                  column_name,
                                                                                  distribution_type, 
                                                                                  features_outlier_limits,
                                                                                  features_in_wrong_distribution)
        
    #p#print(f"features_outlier_limits:\n{features_outlier_limits}")
    
    # Ignore features used in the less appropriate distribution based on sknewness
    for f in features_in_wrong_distribution: 
        features_to_remove_outliers_from.remove(f)

    #------------------
    # Remove outliers #
    #------------------
    removed_outliers_X_t = X_t.copy(deep=True)
    removed_outliers_X_t = treat_outliers(removed_outliers_X_t, outlier_treating_method, 
                                          features_to_remove_outliers_from,
                                          features_outlier_limits)
    
    
    print(f"Outliers successfully removed!\nCurrent X_t dims: {removed_outliers_X_t.shape}")
    print(f"Nº of outliers removed: {X_t.shape[0] - removed_outliers_X_t.shape[0]}"
          f" (0 is fine if the method was 'Capping'!)")
    return removed_outliers_X_t



def encode_dataframe_data(X_t, X_train, y_train, encoding_method, drop_method=None, base_n=16, kfolds=5, smoothing='auto'):
   
    X_t_numbers_no_dates_times, X_t_numbers_dates_times, X_t_objects, X_t_bools, X_t_lists = ums.split_dataframe_data_by_type(X_t)
    X_train_numbers_no_dates_times, X_train_numbers_dates_times, X_train_objects, X_train_bools, X_train_lists = ums.split_dataframe_data_by_type(X_train) 
    
    # Encode non date/time numeric data (not applicable, only here for the sake 
    #  of clarity)
    encoded_X_t_numbers_no_dates_times = X_t_numbers_no_dates_times

    # Encode date/time numeric data
    # Using the periodic functions of sin and cos:
    for column_name in X_t_numbers_dates_times:
        if ums.is_month_column_name(column_name):        
            max_cardinality = ums.get_max_cardinality_of_months()
            
        elif ums.is_day_column_name(column_name):
            max_cardinality = ums.get_max_cardinality_of_days()
            
        X_t_numbers_dates_times['encoded_'+column_name+'_sin'] = np.sin(2*np.pi * X_t_numbers_dates_times[column_name]/max_cardinality)
        X_t_numbers_dates_times['encoded_'+column_name+'_cos'] = np.cos(2*np.pi * X_t_numbers_dates_times[column_name]/max_cardinality)
        # Drop original (and non-cyclic representation of the) date/time column
        X_t_numbers_dates_times.drop(columns=column_name, inplace=True)

    encoded_X_t_numbers_dates_times = X_t_numbers_dates_times

    # Encode strings (/objects)
    match (encoding_method):
        case 1: # OneHotEncoding
            
            # The "unknown labels" issue could be solved in 2 ways:
            #  1 (and most correct?): handle_unknown='ignore' which will
            #    encode the unknown categoricals (test set) as all 0's
            #  2: Merge traint and test set to get all labels in each feature
            #      as a list of lists and then pass that to the 'categories='
            #      parameter.
            #     However, what's the purpose of fitting with x_train if the
            #      encoder will already know all the labels? It will then
            #      encode the not seen ones of the test set with a '1' which
            #      is erroneous and defeats the purpose of fitting with the
            #      test set?
            #      (This number 2 solution was seen at a towardsdatascience
            #       'one-hot-encoding scikit vs pandas' post)
            #
            encoder = OneHotEncoder(drop=drop_method, handle_unknown='ignore', sparse_output=False)
            encoder.fit(X_train_objects)
            encoded_objects = encoder.transform(X_t_objects)
            encoded_objects = pd.DataFrame(index=X_t_objects.index, 
                                           data=encoded_objects,
                                           columns=encoder.get_feature_names_out())

            
        case 2: # Base N Encoding
            encoder = ce.BaseNEncoder(base=base_n) 
            encoder.fit(X_train_objects)
            encoded_objects = encoder.transform(X_t_objects)
            

        case 3: # Target Encoding
            encoder = TargetEncoder(cv=kfolds, smooth=smoothing).set_output(transform="pandas")
            encoder.fit(X_train_objects, y_train)
            encoded_objects = encoder.transform(X_t_objects)
    
    encoded_objects = encoded_objects.add_prefix('encoded_')
    encoded_X_t_objects = encoded_objects
    
    
    # Encode booleans
    encoded_X_t_bools = X_t_bools.astype('int') #False-0, True-1
    encoded_X_t_bools = encoded_X_t_bools.add_prefix('encoded_')
    
    # Encode lists  
    # (In this case, they have already been dropped in
    #  'handle_missing_dataframe_data_X(...)' by "expert knowledge". Reasons
    #  explained there)
    encoded_X_t_lists = X_t_lists
   
    encoded_X_t = pd.concat([encoded_X_t_numbers_no_dates_times, 
                             encoded_X_t_numbers_dates_times, encoded_X_t_objects,
                             encoded_X_t_bools, encoded_X_t_lists],
                            axis=1) 
    print(f"Dims after encoding: {encoded_X_t.shape}")
    return encoded_X_t




def scale_dataframe_data(df_t, df_train, scaling_method):
    # Avoid rescaling the following features:
    #  - encoded categoricals
    #  - dates/times, already encoded with periodic functions 
    
    scaled_df_t = df_t.copy(deep=True)
    scaler = None # In case no feature is numeric (only relevant for target var.) 

    match(scaling_method):
        case 0:
            scaler = 'No scaler'
    
        case 1: # Normalization (Min-max)
            for column_name in scaled_df_t.columns:
                #* Type check needed for when df_t = y_train 
                if not ums.is_encoded_column_name(column_name) and np.issubdtype(df_t[column_name].dtype, np.number): #*
                    minmax_scaler = MinMaxScaler(feature_range=(0,1)) 
                    # Nans are treated as missing values, disregarded during fit
                    #  and simply kept (ignored) during transform
                    #* Pass as dataframe with '[[]]'
                    minmax_scaler.fit(df_train[[column_name]]) #*
                    scaled_df_t[column_name] = minmax_scaler.transform(scaled_df_t[[column_name]]) #*
                    #p#print(f"Min-max of {column_name}:"
                    #        f" max:{scaled_X_t[column_name].nlargest(n=1).values},"
                    #        f" min:{scaled_X_t[column_name].nsmallest(n=1).values}")
            scaler = minmax_scaler                     
    
        case 2: # Mean Normalisation
            for column_name in scaled_df_t.columns:
                if not ums.is_encoded_column_name(column_name) and np.issubdtype(df_t[column_name].dtype, np.number):
                    # Avoid NaN's if min and max are the same.
                    if scaled_df_t[column_name].max() - scaled_df_t[column_name].min() == 0:
                        scaled_df_t[column_name] = 0
                    else:
                        scaled_df_t[column_name] = (scaled_df_t[column_name] - df_train[column_name].mean() ) / \
                                                    ( scaled_df_t[column_name].max() - scaled_df_t[column_name].min() )
                    #p#print(f"Min-max of {column_name}:"
                    #        f" max:{encoded_X_t[column_name].nlargest(n=1).values},"
                    #        f" min:{encoded_X_t[column_name].nsmallest(n=1).values}")
                    #p#print(f"And mean: {encoded_X_train[column_name].mean()}")
            scaler = 'MeanNormalisation'
            
    
        case 3: # Standardization
            for column_name in scaled_df_t.columns:
                if not ums.is_encoded_column_name(column_name) and np.issubdtype(df_t[column_name].dtype, np.number): 
                    standard_scaler = StandardScaler() 
                    standard_scaler.fit(df_train[[column_name]])
                    scaled_df_t[column_name] = standard_scaler.transform(scaled_df_t[[column_name]])
                    #p#print(f"Min-max of {column_name}:"
                    #        f" max:{scaled_X_t[column_name].nlargest(n=1).values},"
                    #        f" min:{scaled_X_t[column_name].nsmallest(n=1).values}")
            scaler = standard_scaler


        case 4: # Robust Scaler
            for column_name in scaled_df_t.columns:
                if not ums.is_encoded_column_name(column_name) and np.issubdtype(df_t[column_name].dtype, np.number): 
                    robust_scaler = RobustScaler() 
                    robust_scaler.fit(df_train[[column_name]])
                    scaled_df_t[column_name] = robust_scaler.transform(scaled_df_t[[column_name]])
                    #p#print(f"Min-max of {column_name}:" 
                    #        f" max:{scaled_X_t[column_name].nlargest(n=1).values},"
                    #        f" min:{scaled_X_t[column_name].nsmallest(n=1).values}")
            scaler = robust_scaler


        case 5: # Power Transformer
            for column_name in scaled_df_t.columns:
                if not ums.is_encoded_column_name(column_name) and np.issubdtype(df_t[column_name].dtype, np.number): 
                    power_transformer = PowerTransformer(method='yeo-johnson', standardize=False)
                    power_transformer.fit(df_train[[column_name]])
                    scaled_df_t[column_name] = power_transformer.transform(scaled_df_t[[column_name]])
                    #print(f"Min-max of {column_name}:"
                    #      f" max:{scaled_X_t[column_name].nlargest(n=1).values},"
                    #      f" min:{scaled_X_t[column_name].nsmallest(n=1).values}")
            scaler = power_transformer

    # Return the scaler instance only if scaling (single target) y
    #  It will be used to revert the scaling
    if df_t.shape[1] == 1:
        return scaled_df_t, scaler
    else:
        return scaled_df_t



def dimensionally_reduce(X_t, X_train, y_train, dimensionality_reduction_method):
    
    match(dimensionality_reduction_method):
        case 1: # Random Forests    
            # Not ready for multi-target
            if y_train.shape[1] > 1:
                print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n"
                      f"(Feature selection via) Random Forests Classification/Regression"
                      f" is not ready for multi target, please select another"
                      f" dim.reduction method\n"
                      f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                return
 
            if np.issubdtype(y_train[y_train.columns[0]].dtype , np.number):
                rf_model = RandomForestRegressor(n_jobs=-1, random_state=42)        
            elif np.issubdtype(y_train[y_train.columns[0]].dtype , object):
                rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)
            rf_model.fit(X_train, y_train.values.ravel())
            
            feature_importances = pd.Series(rf_model.feature_importances_, index=X_t.columns).sort_values(ascending=False)
            
            # Visualize feature importances 
            # Plot#
            '''
            print(f"\nfeat_importances:\n{feature_importances}")
            plot1 = sns.barplot(x=feature_importances[:15], y=feature_importances[:15].index)
            plot1.set_title("Barchart on the top 15 feature importances for a prediction on...")
            plot1.set_xlabel('Feature importance score')
            plot1.set_ylabel('Feature name')
            plt.tight_layout()
            plt.show()
            plot1.figure.savefig(f"rf_feature_importances") 
            '''
            
            # Select features 
            #
            #* Edit theshold at will. Default is 'mean'.
            feature_importance_threshold = 'mean' #*
            rf_model_selector = SelectFromModel(rf_model, threshold=feature_importance_threshold) 
            rf_model_selector.fit(X_train, y_train.values.ravel())
            dimensionally_reduced_X_t = rf_model_selector.transform(X_t)
              
                
        case 2: # PCA
            # Get an initial PCA to find a "good" number of components
            n_pca_components = 'default' 
            if n_pca_components == 'default':
                # Reduce by 1 from min(rows/columns)
                pca = PCA(n_components=min(X_train.shape[0], X_train.shape[1]) - 1) 
            else:
                pca = PCA(n_components=n_pca_components) 
            try:
                pca.fit(X_train)
            except np.linalg.LinAlgError:
                print(f"!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!\n"
                      f"SVD may have failed to converge"
                      f"It is unlikely that the cause is linked to NaN's"
                      f" or 'inf.' values..."
                      f"\nReturning and exiting PCA feature selection"
                      f"\n!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!\n")
                return'N/A'
           
            # Find that number of components by checking when enough variance is
            #  explained
            n_pca_components = None
            explained_variances_ratios = pca.explained_variance_ratio_
            explained_variances_ratios_cumsum = np.cumsum(explained_variances_ratios)

            for i, cumsum in enumerate(explained_variances_ratios_cumsum):
                #If an explained variance ratio of 0.9 is met, leave it at that
                if cumsum >= 0.9 and i<99:
                    n_pca_components = i+1
                    break
                #Otherwise get as many as necessary to "explain" 80% of the data
                elif cumsum >=0.8 and i>=99: #while n_pca_components<100 try to reach 0.9
                    n_pca_components = i+1
                    break
                    
            # Plot for visualization
            '''
            print(f"componen: {n_pca_components}")
            x_limit =100
            plt.title("PCA's explained variance ratios barchart")
            plot1 = sns.barplot(x=range(0, len(explained_variances_ratios[:x_limit])), y=explained_variances_ratios[:x_limit], label='Explained variances ratios', align='center')
            plot1.step(x=range(0, len(explained_variances_ratios_cumsum[:x_limit])), y=explained_variances_ratios_cumsum[:x_limit], label='Explained variances ratios cumul.sum.', where='mid')
            plot1.set_xticks(np.arange(0, x_limit, step=5))
            plot1.set_xlabel("Explained variance ratio index")
            plot1.set_ylabel("Explained variance ratio")
            plot1.legend(loc='best')
            plt.tight_layout()
            plt.show()
            plot1.figure.savefig(f"pca_explained_variance_ratios")
            '''
            print(f"Number of components and corresponding explained variance"
                  f" ratio:\n{n_pca_components} -" 
                  f" {explained_variances_ratios_cumsum[n_pca_components-1]}")
            
            # Build a PCA model with the obtained "recommended" number
            #  of components
            pca = PCA(n_components = n_pca_components)
            pca.fit(X_train)
            
            dimensionally_reduced_X_t = pca.transform(X_t)

    
        case 3: #LDA
            # Not ready for multi-target
            if y_train.shape[1] > 1:
                print(f"!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!!\n"
                      f"LDA is not ready for multi target, please select" 
                      f" another dim.reduction method\n"
                      f"Returning and exiting...\n"
                      f"!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!\n")
                return 
                
            # Not meant for numeric "labels"
            if not np.issubdtype(y_train[y_train.columns[0]].dtype, object):
                print(f"!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!\n"
                      f"The target variable is of type:"
                      f" {y_train[y_train.columns[0]].dtype}\n"
                      f"LDA accepts only the feature/column type: \"object\".\n" 
                      f"Returning and exiting...\n"
                      f"!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!\n")
                return 'N/A'
                
            num_predictors = X_train.shape[1] # Number features            
            #* Number of labels of the target
            num_target_classes = y_train[y_train.columns[0]].nunique() #*
            lda = LinearDiscriminantAnalysis(n_components=min(num_predictors, num_target_classes-1))

            try:
                lda.fit(X_train, y_train.values.ravel())
            except np.linalg.LinAlgError:
                print(f"!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!\n"
                      f"SVD may have failed to converge"
                      f"It is unlikely that the cause is linked to NaN's or"
                      f" 'inf.' values..."
                      f"\nReturning and exiting LDA feature selection"
                      f"!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!!!\n")
                return'N/A'
            
            dimensionally_reduced_X_t = lda.transform(X_t)
    
            # Plot #
            '''
            explained_variances_ratios = lda.explained_variance_ratio_
            print(f"a\n {explained_variances_ratios}")
            explained_variances_ratios_cumsum = np.cumsum(explained_variances_ratios)
            plt.title("LDA's explained variance ratios barchart")
            plot1 = sns.barplot(x=range(0, len(explained_variances_ratios)), y=explained_variances_ratios, label='Explained variances ratios', align='center')
            plot1.step(x=range(0, len(explained_variances_ratios_cumsum)), y=explained_variances_ratios_cumsum, label='Explained variances ratios cumul.sum.', where='mid')
            plot1.set_xticks(np.arange(0, len(explained_variances_ratios), step=5))
            plot1.set_xlabel("Explained variance ratio index")
            plot1.set_ylabel("Explained variance ratio")
            plot1.legend(loc='best')
            plt.tight_layout()
            plt.show()
            #plot1.figure.savefig(f"lda_explained_variance_ratios")
            '''

    
        case 4: #Truncated SVD
            # Choose the number of components based on plotted explained variance:
            #  - Build a T_SVD with all the features to see the number of them
            #    we need to get the desired explained variance
            # Note: gives at most min(rows, columns) components (so no point
            #       putting anything above that in the hyerparameter)
            analysis_truncated_svd = TruncatedSVD(n_components=min(X_train.shape[0], X_train.shape[1]), algorithm='randomized', random_state=42)
            analysis_truncated_svd.fit(X_train)
        
            explained_variance_percent = analysis_truncated_svd.explained_variance_ratio_ * 100 
            explained_variance_percent_cumsum = np.cumsum(explained_variance_percent)
            
            # Plot 1 #
            # Explained variance and comulative sum WITH intermidiate numbers
            #  being displayed
            '''
            plt.bar(range(1, min(X_train.shape[0], X_train.shape[1])+1), explained_variance_percent, align='center', label='Explained variance by component (%)', color='lightblue')
            # Add numbers in the graph at each (x,y) for clarity 
            for i in range(1, min(X_train.shape[0], X_train.shape[1])+1, 2):
                plt.text(i, explained_variance_percent[i-1]*1.01, round(explained_variance_percent[i-1], 2), ha='center', 
                            bbox=dict(facecolor='yellow', boxstyle=BoxStyle('square', pad=0.2), alpha=0.8))  
            
            plt.step(range(1, min(X_train.shape[0], X_train.shape[1])+1), explained_variance_percent_cumsum, label='Cumulative explained variance', color='darkblue')
            # Add numbers in the graph at each (x,y) for clarity
            for i in range(1, min(X_train.shape[0], X_train.shape[1])+1, 2):
                plt.text(i, explained_variance_percent_cumsum[i-1]*1.01, round(explained_variance_percent_cumsum[i-1], 2), ha='center',
                        bbox=dict(facecolor='orange', boxstyle=BoxStyle('square', pad=0.2), alpha=0.8)) 
            
            plt.xlabel('Component index')
            plt.ylabel('Explained variance percentage')
            plt.xticks(ticks=list(range(1, X_train.shape[1]+1)))
            plt.legend(loc='best') 
            plt.tight_layout() 
            plt.show()
             
            '''
           
            # Plot 2 #
            # Simplified explained variance and comulative sum WITHOUT 
            #  intermidiate numbers being displayed
            '''
            explained_variances_ratios = analysis_truncated_svd.explained_variance_ratio_
            explained_variances_ratios_cumsum = np.cumsum(explained_variances_ratios)

            plt.title("Truncated SVD's explained variance ratios barchart")
            plot2 = sns.barplot(x=range(0, len(explained_variances_ratios)), y=explained_variances_ratios, label='Explained variances ratios', align='center')
            plot2.step(x=range(0, len(explained_variances_ratios_cumsum)), y=explained_variances_ratios_cumsum, label='Explained variances ratios cumul.sum.', where='mid')
            plot2.set_xticks(np.arange(0, len(explained_variances_ratios), step=5))
            plot2.set_xlabel("Explained variance ratio index")
            plot2.set_ylabel("Explained variance ratio")
            plot2.legend(loc='best')
            plt.tight_layout()
            plt.show()
            #plot2.figure.savefig(f"truncated_SVD_explained_variance_ratios")
            '''

            # Select number of components based on threshold of explained 
            #  variance (ratio (*100)) 
            explained_variance_percent_threshold = 99
            n_necessary_components = 0 
            for i in range(len(explained_variance_percent_cumsum)):
                if explained_variance_percent_cumsum[i] >= explained_variance_percent_threshold:
                    n_necessary_components = i + 1
                    break

            # Now reduce the dimensionality based on the selected
            #  features/components
            truncated_svd = TruncatedSVD(n_components=n_necessary_components, 
                                         algorithm='randomized', random_state=42)
            truncated_svd.fit(X_train)
            dimensionally_reduced_X_t = truncated_svd.transform(X_t)

    
        case 5: #Kernel PCA
            kernel_pca = KernelPCA(n_components=100, kernel='poly', random_state=42, n_jobs=-1)
            kernel_pca.fit(X_train)
            dimensionally_reduced_X_t = kernel_pca.transform(X_t)
            
            # Plot #
            '''
            plt.title("Explained variance ratios barchart")
            plot1 = sns.barplot(x=range(0, len(kernel_pca.eigenvalues_)), y=kernel_pca.eigenvalues_, align='center')
            #plot1.step(x=range(0, len(kernel_pca.eigenvalues_)), y=explained_variances_ratios_cumsum[:x_limit], label='Explained variances ratios cumul.sum.', where='mid')
            plot1.set_xticks(np.arange(0, len(kernel_pca.eigenvalues_), step=5))
            plot1.set_xlabel("Eigenvalue index")
            plot1.set_ylabel("Eigenvalues")
            #plot1.legend(loc='best')
            plt.tight_layout()
            plt.show()
            plot1.figure.savefig(f"kernel_pca_top100_eigenvalues")
            '''
           
            
    return dimensionally_reduced_X_t


##########################
### Data preprocessing ###
##########################
def preprocess_dataframes_data(dfs_to_merge, preprocessing_settings):
    
    #############
    # Settings: #
    #############
    #
    # Default target for one-off predictions
    target_variable_name = preprocessing_settings['target_variable_name'] 
    #* For selecting which variables will be filled
    missing_data_threshold = preprocessing_settings['missing_data_threshold'] #*
    features_to_drop_expert_knowledge = preprocessing_settings['features_to_drop_expert_knowledge']
    #* For splitting train/test data based on the Nan's of the target 
    #  (fill missing data with predictions)
    is_nan_splitting = preprocessing_settings['is_nan_splitting']#*
    numeric_imputation_method_X = preprocessing_settings['numeric_imputation_method_X'] 
    categorical_imputation_method_X = preprocessing_settings['categorical_imputation_method_X']
    numeric_imputation_method_y = preprocessing_settings['numeric_imputation_method_y']
    categorical_imputation_method_y = preprocessing_settings['categorical_imputation_method_y']
    distribution_type = preprocessing_settings['distribution_type']
    outlier_treating_method = preprocessing_settings['outlier_treating_method']
    encoding_method = preprocessing_settings['encoding_method']
    scaling_method = preprocessing_settings['scaling_method']
    dimensionality_reduction_method = preprocessing_settings['dimensionality_reduction_method'] 

    
    ###############
    # Data Cleaning 
    ###############
    #
    # There are no equivalent tables in the scrapped datasets to "group" or
    #  remove typos, different spelling, value-formatting, etc... 
    #
    # The non-existence of duplicates was ensured by SQL 'ON UPDATE' clauses
    #  during the insertion
    #
    # No erroneous data was originated from the scrapping
    #
    # Let's ensure the variables are in the right types
    #
    cleaned_dfs_to_merge = clean_dataframes_data(dfs_to_merge)

    ##################
    # Data Integration
    ##################
    #
    # Merge the data from the web scrapped tables to create the "full" dataset
    #
    integrated_df = integrate_dataframes_data(cleaned_dfs_to_merge) 
        
    ############################################
    #Split the data into training and test sets:
    ############################################

    X_train, X_test, y_train, y_test = split_dataframe_data_train_test(integrated_df,
                                                                       target_variable_name,
                                                                       is_nan_splitting)
    print(f"Initial dims: X_train: {X_train.shape}, X_test: {X_test.shape},"
          f"y_train: {y_train.shape}, y_test: {y_test.shape}")
    
    ##############################################################
    # Handle (impute/remove) missing data (before training models)
    ##############################################################
    #
    # Any feature columns that are not deemed useful based on expert knowledge
    #  can be instantly removed
    # For now, those will consist of:
    #  - 'real_estate_agent_id': Home_liverpool_id is for the most part
    #                            connected with the listing date, so it
    #                            might be a useful source of information
    #                            for the algorithm for predictions, being
    #                            added as an exception to the 
    #                            is_id_column_name() method). The estate 
    #                            agent id, on the other hand, isn't, so it
    #                            can be discarded.
    #
    #  - 'photos': list of urls which would have to be encoded and most likely 
    #              with no value
    #
    #  - 'feature_set': given the nature of the listings, this list can contain
    #                   features written in a completely arbitrary way, leading
    #                   to even similar features having different encodings later
    #                   on in the preprocessing steps. Adding to the apparent
    #                   impracticality of encoding a list, the result could also
    #                   be misleading for the model.
    #
    #  - 'description': (similar reasoning to 'feature_set')
    #
    #
    # Features with missing data ratio over a certain threshold will be removed
    #
    # Datetime and bools had a NOT NULL clause in the databse, hence not needed
    # This leaves Numerical and Categorical (numpy object) data
    
    for column_name in X_train.columns:
        if ums.is_id_column_name(column_name):
            features_to_drop_expert_knowledge.add(column_name)


    if is_nan_splitting == True:
        if numeric_imputation_method_X == 'DEL':
            print(f"WARNING: numeric_imputation_method_X will be changed from"
                  f" 'DEL' to 1 to avoid y_test row losses.")
            numeric_imputation_method_X = 1
        if categorical_imputation_method_X == 'DEL':
            print(f"WARNING: categorical_imputation_method_X will be changed from"
                  f" 'DEL' to 'A' to avoid y_test row losses.")
            categorical_imputation_method_X = 'A'  

    # X imputation/deletion
    imputed_X_train = handle_missing_dataframe_data_X(
        X_train, X_train, missing_data_threshold, features_to_drop_expert_knowledge,
        numeric_imputation_method_X, categorical_imputation_method_X
    )
    imputed_X_test = handle_missing_dataframe_data_X(
        X_train, X_test, missing_data_threshold, features_to_drop_expert_knowledge,
        numeric_imputation_method_X, categorical_imputation_method_X
    )   
    # If deletion method, update target (y) to match X!
    if numeric_imputation_method_X == 'DEL' or categorical_imputation_method_X == 'DEL':
        y_train = y_train.loc[imputed_X_train.index.to_list()]
        y_test = y_test.loc[imputed_X_test.index.to_list()] 

    # y imputation/deletion #
    y_train = handle_missing_dataframe_data_y(y_train, numeric_imputation_method_y,
                                              categorical_imputation_method_y)
    # No imputation method should be used on the testing target, simply drop Nans.
    if is_nan_splitting == False:
        y_test.dropna(inplace=True) 
    
    # If deletion method, update predictor (X) to match y! #
    if numeric_imputation_method_y == 'DEL' or categorical_imputation_method_y == 'DEL':
        imputed_X_train = imputed_X_train.loc[y_train.index.to_list()]
    # Outside of the 'if' since y_test drops its "NaN"s rehardless
    imputed_X_test = imputed_X_test.loc[y_test.index.to_list()]  


    ########################
    # Detect/Remove Outliers
    ########################    

    # Edit and add as the 4th parameter of `remove_outliers()` if only a
    #  specific set of features is to have outliers treated.
    #  By default, all numeric ones will be adressed 
    features_to_remove_outliers_from = [] 
    removed_outliers_X_train = remove_outliers(imputed_X_train, distribution_type,
                                               outlier_treating_method) 
    
    removed_outliers_X_test = imputed_X_test
    
   
    # Plotting distributions before/after outlier removal
    '''
    for column_name in removed_outliers_X_train.columns:
        if np.issubdtype(removed_outliers_X_train[column_name].dtype, np.number):
            plt.figure(figsize=(10,10))
            plt.subplot(2,2,1)
            plt.title(f"{column_name} value distribution")
            sns.histplot(imputed_X_train[column_name])
            plt.subplot(2,2,2)
            sns.boxplot(data=imputed_X_train[column_name])
            #---------------------------------------------------
            plt.subplot(2,2,3)
            plt.title(f"{column_name} value distribution (w/o outliers, method: {outlier_treating_method}")
            sns.histplot(removed_outliers_X_train[column_name])
            plt.subplot(2,2,4)
            sns.boxplot(data=removed_outliers_X_train[column_name])

            plt.subplots_adjust(hspace=0.4)
            plt.show() 

    '''
    
    # Update target (y)
    # Remove any deleted rows from the target variable as well 
    # Note: Indexes MUST be PRESERVED so they can MATCH 
    y_train = y_train.loc[removed_outliers_X_train.index.to_list()]
    y_test = y_test.loc[removed_outliers_X_test.index.to_list()]


    #####################################################
    # Encode the (categorical, boolean and datetime) data
    #####################################################
    # Cyclic Feature Encoding will be used on cyclic features extracted from
    #  the original datetime feature. In this case, month and day will be encoded
    #  to preserve the relationship between the "extreme" values (month 1<->12, 
    #  day 2nd<->28th etc 
    #
    # Booleans will simply be converted to type 'int'
    


    encoded_X_train = encode_dataframe_data(removed_outliers_X_train,
                                            removed_outliers_X_train,
                                            y_train, encoding_method)
    encoded_X_test = encode_dataframe_data(removed_outliers_X_test,
                                           removed_outliers_X_train,
                                           y_train, encoding_method)
   
    
    #################
    #Feature Scaling
    #################

    #Values to retain to revert the scaling of numeric predictions:
    y_test_mins, y_test_maxs, y_train_means = ums.get_values_for_scaling_reversion(y_test,
                                                                                   y_train)
    

    # Scale target variable(s) (if applicable) and get scaler to revert it on the
    #  prediction results
    if ums.has_numeric_feature(y_train): 
        # Scale test first else (scaled)y_train will be used in it fitting due
        #  to same var name ':)
        #
        # 'scaler' gets assigned on both these 'y_' since the scaling function
        #  is set to always return it. It's not an issue in any case, since 
        #  it's the same. 
        y_test, scaler = scale_dataframe_data(y_test, y_train, scaling_method) 
        y_train, scaler = scale_dataframe_data(y_train, y_train, scaling_method) 
    else:
        scaler = None

    # Scale predictors (if applicable)
    if ums.has_numeric_feature(encoded_X_train): 
        scaled_X_train = scale_dataframe_data(encoded_X_train, encoded_X_train,
                                              scaling_method)
        scaled_X_test = scale_dataframe_data(encoded_X_test, encoded_X_train,
                                             scaling_method)
    else:
        scaled_X_train = encoded_X_train
        scaled_X_test = encoded_X_test
        
    scaler_info = [scaler, y_test_mins, y_test_maxs, y_train_means]


    ############################################
    # Dimensionality Reduction
    #
    # (Feature Selection and Feature Projection)
    ############################################

    dimensionally_reduced_X_train = dimensionally_reduce(scaled_X_train,
                                                         scaled_X_train,
                                                         y_train,
                                                         dimensionality_reduction_method)
    dimensionally_reduced_X_test = dimensionally_reduce(scaled_X_test,
                                                        scaled_X_train,
                                                        y_train,
                                                        dimensionality_reduction_method)
    # Variable could be of type 'str' if there was an 'N/A' return
    #  due to, for instance, a failure of the model to converge
    #  with the current parameters
    if type(dimensionally_reduced_X_train) is not str: 
        print(f"dimensionally_reduced_X_train shape:\n{dimensionally_reduced_X_train.shape}")
    
    return (dimensionally_reduced_X_train, dimensionally_reduced_X_test,
            y_train, y_test, scaler_info)


